In [1]:
#https://towardsdatascience.com/web-scraping-craigslist-a-complete-tutorial-c41cea4f4981
#Importing all of the required packages
from requests import get
from bs4 import BeautifulSoup as soup
import numpy as np
from time import sleep
import re
from random import randint #avoid throttling by not sending too many requests one after the other
from warnings import warn
from time import time
import pandas as pd

In [2]:
#Url that we want to scrape
url = "https://sfbay.craigslist.org/search/sfc/apa?hasPic=1"

In [3]:
response = get(url)
page_soup = soup(response.text, "html.parser")

#get the html container of the housing posts
posts = page_soup.find_all('li', class_= 'result-row')

In [19]:
#Let's get the neighbourhood
hood = posts[0].find('span', class_ = 'result-hood')
hood.text.strip().replace('(','').replace(')','')

'potrero hill'

In [28]:
#Let's get the price
price = posts[0].find('span', class_ = 'result-price')
price.text

'$3871'

In [197]:
#Let's get bed, bath and sqft
bedding = posts[21].find('span', class_ = 'housing')


if bedding is None:
    number_bedroom = ''
    sqft = ''
else:
    test = bedding.text.strip().split(" ")
    testlist = ' '.join(test).split()
    
    if len(testlist) == 4:
        number_bedroom = testlist[0]
        sqft = testlist[2]
    
    elif testlist[0][-2:] == 'br':
        number_bedroom = testlist[0]
        sqft = ''
    
    elif testlist[0][-3:] == 'ft2':
        number_bedroom = ''
        sqft = testlist[0]
            
#test = bedding.text.strip().split(" ")
#testlist = ' '.join(test).split()
#number_bedroom = testlist[0]
#number_bathroom = testlist[1]
#sqft = testlist[2]
#print(test)
#print(testlist)
print(number_bedroom, sqft)

In [4]:
#Build the loop on all the listings and on all the pages
results_num = page_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_='totalcount').text)
pages = np.arange(0, results_total+1, 120)

In [6]:
#initializing some of the arrays we will need once we loop
iterations = 0

post_timing = []
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []

In [8]:
for page in pages:

    #get requests
    response = get("https://sfbay.craigslist.org/search/sfc/apt?"
                   + "s=" #the parameter for defining the page number
                   + str(page) #the page number in the pages array from earlier
                   + "&hasPic=1"
                   + "&availabilityMode=0")


    sleep(randint(1,5))

    #throw warning for status codes that are not 200
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    #define the html text
    page_html = soup(response.text, 'html.parser')

    #define the posts
    posts = page_html.find_all('li', class_= 'result-row')

    for post in posts:
        if post.find('span', class_ = 'result-hood') is not None:
            #posting date
            #grab the datetime element 0 for date and 1 for time
            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)

            #post link
            post_link = post_title['href']
            post_links.append(post_link)

            #removes the \n whitespace from each side, removes the currency symbol, and turns it into an int
            post_price = int(post.a.text.strip().replace("$", ""))
            post_prices.append(post_price)

            if post.find('span', class_ = 'housing') is not None:

                #if the first element is accidentally square footage
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:

                    #make bedroom nan
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)

                    #make sqft the first element
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)

                #if the length of the housing details element is more than 2
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:

                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)

                    #and sqft will be number 3, so set these here and append
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)

                #if there is num bedrooms but no sqft
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:

                    #therefore element 0 will be bedroom count
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)

                    #and sqft will be number 3, so set these here and append
                    sqft = np.nan
                    sqfts.append(sqft)

                else:
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)

                    sqft = np.nan
                    sqfts.append(sqft)

                #if none of those conditions catch, make bedroom nan, this won't be needed
            else:
                bedroom_count = np.nan
                bedroom_counts.append(bedroom_count)

                sqft = np.nan
                sqfts.append(sqft)
                #    bedroom_counts.append(bedroom_count)

                #    sqft = np.nan
                #    sqfts.append(sqft)

    iterations += 1
    print("Page " + str(iterations) + " scraped successfully!")

print("\n")
print("Scrape complete!")

Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!
Page 15 scraped successfully!
Page 16 scraped successfully!
Page 17 scraped successfully!
Page 18 scraped successfully!
Page 19 scraped successfully!
Page 20 scraped successfully!
Page 21 scraped successfully!
Page 22 scraped successfully!
Page 23 scraped successfully!
Page 24 scraped successfully!
Page 25 scraped successfully!
Page 26 scraped successfully!


Scrape complete!


In [9]:
eb_apts = pd.DataFrame({'posted': post_timing,
                       'neighborhood': post_hoods,
                       'post title': post_title_texts,
                       'number bedrooms': bedroom_counts,
                        'sqft': sqfts,
                        'URL': post_links,
                       'price': post_prices})

In [10]:
print(eb_apts.info())
eb_apts.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2868 entries, 0 to 2867
Data columns (total 7 columns):
posted             2868 non-null object
neighborhood       2868 non-null object
post title         2868 non-null object
number bedrooms    0 non-null float64
sqft               0 non-null float64
URL                2868 non-null object
price              2868 non-null int64
dtypes: float64(2), int64(1), object(4)
memory usage: 156.9+ KB
None


,posted,neighborhood,post title,number bedrooms,sqft,URL,price
0,2019-04-25 18:30,(san rafael),Subaru Outback 2008 Fender drivers side plus f...,NaN,NaN,https://sfbay.craigslist.org/nby/pts/d/san-raf...,175
1,2019-04-25 18:30,(san rafael),Subaru Outback 2008 Fender drivers side plus f...,NaN,NaN,https://sfbay.craigslist.org/nby/pts/d/san-raf...,175
2,2019-04-25 18:30,(cupertino),3TB Seagate SkyHawk Surveillance Hard Drive ST...,NaN,NaN,https://sfbay.craigslist.org/sby/sop/d/sunnyva...,65
3,2019-04-25 18:30,(+ CAR EXPO AUTO CENTER),2013 Honda Accord EX-L v6 -TOP $$$ FOR YOUR TR...,NaN,NaN,https://sfbay.craigslist.org/eby/ctd/d/sacrame...,10979
4,2019-04-25 18:30,(richmond / point / annex),"Cuisinart Electric Knife, $18",NaN,NaN,https://sfbay.craigslist.org/eby/for/d/cuisina...,18
5,2019-04-25 18:30,(milpitas),White table,NaN,NaN,https://sfbay.craigslist.org/sby/fuo/d/san-jos...,0
6,2019-04-25 18:30,(Tracy),Fisher Marine aluminum Honda outboard,NaN,NaN,https://sfbay.craigslist.org/sby/boa/d/banta-f...,2500
7,2019-04-25 18:30,(san francisco),Large Mirror 5'x4',NaN,NaN,https://sfbay.craigslist.org/sfc/hsh/d/san-fra...,100
8,2019-04-25 18:30,(santa cruz),Men's O'Neill Wetsuit Heat,NaN,NaN,https://sfbay.craigslist.org/scz/spo/d/santa-c...,200
9,2019-04-25 18:30,(napa county),Beanie Babies - PRISTINE WITH TAGS!,NaN,NaN,https://sfbay.craigslist.org/nby/clt/d/napa-be...,2500
